1.파이썬-> 구글스프레드시트로 연결시켜서 옮기기

2.사이트에서 크롤링

3.조건 1.OWNER / 2.has image / 3. Price 5,000이하

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = [
'https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive',
]

#json파일
json_file_name = 'sunlit-cove-371001-e6ac09769edd.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#URL가져오기
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/10lPp04ItUu7w0Lgt3gRXD_rvtgW8ZLtbfjRcwnmnkyI/edit?pli=1#gid=0'
doc = gc.open_by_url(spreadsheet_url)

Page_i = 1
sheet_name = "page " + str(Page_i)
ws = doc.add_worksheet(title=sheet_name, rows=150, cols=26)
worksheet = doc.worksheet(sheet_name) # 시트선택
worksheet.update_acell('A1', 'DATE')
worksheet.update_acell('B1', 'LOCATION')
worksheet.update_acell('C1', 'TITLE')
worksheet.update_acell('D'+str(1), 'IMAGE')

In [ ]:
#마무리
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests

#Gspread 연동
scope = [
'https://spreadsheets.google.com/feeds',
'https://www.googleapis.com/auth/drive',
]

#json파일
json_file_name = 'sunlit-cove-371001-e6ac09769edd.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#URL가져오기
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/10lPp04ItUu7w0Lgt3gRXD_rvtgW8ZLtbfjRcwnmnkyI/edit?pli=1#gid=0'
doc = gc.open_by_url(spreadsheet_url)


Page_i = 1
sheet_name = "page " + str(Page_i)

ws = doc.add_worksheet(title=sheet_name, rows=150, cols=26)#시트생성

worksheet = doc.worksheet(sheet_name)#시트선택
worksheet.update_acell('A1', 'DATE')
worksheet.update_acell('B1', 'LOCATION')
worksheet.update_acell('C1', 'TITLE')
worksheet.update_acell('D1', 'IMAGE')

driver = webdriver.Chrome()
driver.get('https://sfbay.craigslist.org/search/cta?hasPic=1&max_price=5000&purveyor=owner#search=1~gallery~0~0')
sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


#차량이름, 위치, 날짜
title = soup.select('ol li.cl-search-result div.title-blob')# #은 아이디, 띄어쓰기 하위 속석 .은 class
created_date_and_location = soup.select('ol div.meta')
title_size = len(title)
for i in range(title_size):
    title_text = title[i].text
    split = created_date_and_location[i].text.split('·')
    created_date = split[0]
    location = split[1]
    first_link = soup.select('ol div a.main')
    first_link_size = len(first_link)
    #print('created_date:',created_date,'location:',location,'title:',title_text)
    print(i+2)
    print('title_text:',title_text)
    print('location:',location)
    print('date:',created_date)
    print()



In [ ]:
    try : 
        worksheet.update_acell('A'+str(i+2), created_date)
    except:
        print('예외가 발생했습니다1상.')
        row_data = worksheet.row_values(1)
        sleep(10)
        row_data = worksheet.row_values(1)
        print(row_data)
        worksheet.update_acell('A'+str(i+2), created_date)
        print('예외가 발생했습니다1하.')
    try : 
        worksheet.update_acell('B'+str(i+2), location)
    except Exception as e:
        print(e)
        print('예외가 발생했습니다2상.')
        sleep(10)
        worksheet.update_acell('B'+str(i+2), location)
        print('예외가 발생했습니다2하.')
    try : 
        worksheet.update_acell('C'+str(i+2), title_text)
    except:
        print('예외가 발생했습니다3상.')
        sleep(10)
        worksheet.update_acell('C'+str(i+2), title_text)
        print('예외가 발생했습니다3하.')

In [ ]:
    try :
        for i in range(first_link_size):
            print((first_link[i])['href'])
            res = requests.get((first_link[i])['href'])
            soup2 = BeautifulSoup(res.content, 'html.parser')
    except:
        print('예외발생')

In [ ]:
#해당 게시글에서 사진링크
import requests
from bs4 import BeautifulSoup

res = requests.get('https://sfbay.craigslist.org/pen/cto/7568245359.html')
soup2 = BeautifulSoup(res.content, 'html.parser')
#print(soup2)
soup2_select = soup2.select('#thumbs a.thumb')
for i in range (len(soup2_select)):
    print((soup2_select[i])['href'])

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep


driver = webdriver.Chrome()
driver.get('https://orangecounty.craigslist.org/search/cta#search=1~gallery~0~0')
sleep(1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup)
#elements = driver.find_elements_by_css_selector('#search-results-page-1 > ol')
#보통 html만 사용하여 페이지를 다 표현하지 않고, css로 동적인 이미지 표현한 것도 있어서 
#단순히 html 파싱으로는 가져오는게 한계가 있다.


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import json
url = 'https://sapi.craigslist.org/web/v7/postings/search/full?batch=103-0-360-0-0&cc=US&lang=en&searchPath=cta'

response = requests.get(url)
bsObj = bs(response.content,'html.parser')
jsonObj = response.json()
#print(bsObj)
#print(jsonObj)
json_formatted_str = json.dumps(jsonObj, indent=2)

print(json_formatted_str)
#print(jsonObj["items"])

In [ ]:
print(jsonObj["data"]["decode"]["locationDescriptions"])

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import json
url = 'https://sapi.craigslist.org/web/v7/postings/search/full?batch=103-0-360-0-0&cc=US&hasPic=1&lang=en&max_price=5000&purveyor=owner&searchPath=cta'

response = requests.get(url)
print(response)
jsonObj = response.json()
#print(response.headers)
header = {'Last-Modified': 'Tue'}
#print(bsObj)
#print(jsonObj)
response = requests.get(url, headers = header)
jsonObj = response.json()
#print(jsonObj)
json_formatted_str = json.dumps(jsonObj, indent=2)
#print(json_formatted_str)
print(jsonObj['data']['items'])
#print(jsonObj["items"])